# Imports

In [1]:
import bruce, numpy as np, matplotlib.pyplot as plt, os
from astropy.table import Table, Column
from astropy.stats import sigma_clip
from astropy.time import Time , TimeDelta
from astropy import units as u
import gc, multiprocess
from scipy.stats import median_abs_deviation

/Users/sam/anaconda3/lib/python3.9/site-packages/lightkurve/config/__init__.py:119: UserWarning: The default Lightkurve cache directory, used by download(), etc., has been moved to /Users/sam/.lightkurve/cache. Please move all the files in the legacy directory /Users/sam/.lightkurve-cache to the new location and remove the legacy directory. Refer to https://docs.lightkurve.org/reference/config.html#default-cache-directory-migration for more information.
  warnings.warn(


In [2]:
gc.collect()

0

# Load the tics

Data should have the form of (ascii or CSV, your choice). t_zero is in full BJD, width is in days, depth is in normalised flux. If you ue SPOCFIT, if you fit a single transit you will see 3 reported values on the bottom row which are these values for each event (t_zero, width, depth).

tic_id,	t_zero_1,	width_1,	depth_1,	t_zero_2,	width_2,	depth_2,

In [3]:
duos = Table.read('duos.csv')
duos['tic_id'] = duos['TIC ID']
print(duos)

  TIC ID  Linked TIC ID TOI ID   t_zero_1  ... SG1 Published?   tic_id 
--------- ------------- ------ ----------- ... --- ---------- ---------
 20904104      20904104      - 2458390.178 ...  --         --  20904104
 22317640      22317640      - 2458589.378 ...  --         --  22317640
 42428568      42428568      -  2458475.69 ...  --         --  42428568
      ...           ...    ...         ... ... ...        ...       ...
  5267885       5267885      - 2459012.449 ...  --         --   5267885
286969201     286969201      - 2459018.746 ...  --         -- 286969201
103095888     103095888      - 2459015.816 ...  --         -- 103095888
Length = 59 rows


# Now the main worker function

This function does the following:
-  Load the latest TESS data
-  Flatten the lightcurve
-  Fit the events
-  Calcualte the aliases
-  Plot the permissable aliases
-  Create a report

In [4]:
def func(i):    
    # Create the output dir (we'll use this as cache for the data too)
    output_dir = os.getcwd() + '/{:}'.format(duos['tic_id'][i])
    os.system('mkdir -p {:}'.format(output_dir))
    #if os.path.isfile(output_dir + '/' + 'TIC-{:}_ALIASES.png'.format(duos['tic_id'][i])) : return

    # Now load the TESS data (SPOC, QLP)
    # We are not making our own here like TESSTPF, not yet anyway...
    # for data_type
    #   single_product -> all sectors together
    #   per_sector -> list of per-sector lightcurves
    #   northern_duos -> YEARS 2 and 4, then a list of other sectors 
    #   southern -> YEARS 1 and 3, then a list of other sectors (NOT IMPLEMENTED YET) 
    t, data,data_labels, base_dir =  bruce.ambiguous_period.download_tess_data(duos['tic_id'][i], 
                                                              max_sector=None, 
                                                                   use_ffi=True, 
                                                                   download_dir=None, 
                                                                   bin_length=0.5/24)
    
    # Now flatten the data
    for j, k in zip(data, data_labels):
        # Flatten the data by SG filter, we need an odd kernel legth based on cadence
        j.flatten_data_old(window_width=3, sigmaclip=3, dx_lim=0.1)


#         for seg in bruce.data.find_nights_from_data(j.time, dx_lim=0.2):
#             j.w = np.ones(j.time.shape[0])*np.median(j.flux)

        # Optinally save the data
        j.write_data(output_dir + '/' +'TESS_DATA_{:}.txt'.format(k))
        fig, ax = j.plot_segments(dx_lim=0.5)
        fig.savefig(output_dir + '/' + 'TESS_DATA_{:}.png'.format(k))
        plt.close(fig)
        
        
    # Now we need to adjust the time for TTV cases
    if duos['tic_id'][i]==118339710:
        
        x = np.array([1525.64728,2249.54431,2973.53737,3697.59768])+2457000
        y = np.array([0,-0.087396,-0.078762,-0.002878])
        p_ttv = np.polyfit(x,y, 3)
        def p_ttv(x):
            return 0.08*np.cos(1425.64728 + 2*x*np.pi/(2*(2171.9503999999997)))
        
        for d in data:
            #print(np.polyval(p, d.time))
            #d.time = d.time - np.polyval(p_ttv, d.time)
            d.time = d.time + p_ttv(d.time)
    # # Now re-order_datasets based on epochs given
    # We will unpack now, (data with transits, and data without)
    # We may need to change this for it to work properly (Sam is working on it)
    # Its worth noting we can incorparate ground based data here too
    # data_from_ground = bruce.ambiguous_period.mono_event.photometry_time_series(time, flux, flux_err, w = norm_model)
    # Then this can go in data_other_sectors
    data, data_labels = bruce.ambiguous_period.group_data_by_epochs(data, data_labels, duos['t_zero_1'][i], duos['t_zero_2'][i])
    data, data_other_sectors = data[0], data[1:]



    ############################
    # FIT EVENT 1
    ############################
    # Mask data and create the mono_event object
    nmask = 3
    mask1 = (data.time > (duos['t_zero_1'][i] - nmask*duos['width_1'][i])) &  (data.time < (duos['t_zero_1'][i] + nmask*duos['width_1'][i]))
    #data_event_1 = bruce.ambiguous_period.photometry_time_series(data.time[mask1], data.flux[mask1], data.flux_err[mask1], w=data.w[mask1]) #np.percentile(data.flux[mask1], 50)*np.ones(data.time[mask1].shape[0])
    data_event_1 = bruce.ambiguous_period.photometry_time_series(data.time[mask1], data.flux[mask1], median_abs_deviation(data.flux[mask1])*np.ones(mask1.sum(), dtype=np.float64), w=data.w[mask1]) #np.percentile(data.flux[mask1], 50)*np.ones(data.time[mask1].shape[0])
    m1 = bruce.ambiguous_period.mono_event(duos['t_zero_1'][i], duos['width_1'][i], duos['depth_1'][i], data_event_1, name='TIC-{:}'.format(duos['tic_id'][i]), median_bin_size = None,convolve_bin_size = None)
    
    # Fit the event and report plots
    fig_initial, ax_initial, fig_final, ax_final, return_data_1 = m1.fit_event_with_fixed_period(fit_period=30., plot=True, )
    fig_initial.tight_layout()
    fig_final.tight_layout()
    fig_initial.savefig(output_dir + '/' + 'TIC-{:}_EVENT_1_INITIAL_INITIAL.png'.format(duos['tic_id'][i]))
    fig_final.savefig(output_dir + '/' + 'TIC-{:}_EVENT_1_INITIAL_FINAL.png'.format(duos['tic_id'][i]))
    plt.close(fig_initial); plt.close(fig_final)


    ############################
    # FIT EVENT 2
    ############################
    # Mask data and create the mono_event object
    mask2 = (data.time > (duos['t_zero_2'][i] - nmask*duos['width_2'][i])) &  (data.time < (duos['t_zero_2'][i] + nmask*duos['width_2'][i]))
    data_event_2 = bruce.ambiguous_period.photometry_time_series(data.time[mask2], data.flux[mask2], data.flux_err[mask2], w=data.w[mask2]) #np.percentile(data.flux[mask2], 50)*np.ones(data.time[mask2].shape[0])
    data_event_2 = bruce.ambiguous_period.photometry_time_series(data.time[mask2], data.flux[mask2], median_abs_deviation(data.flux[mask2])*np.ones(mask2.sum(), dtype=np.float64), w=data.w[mask2]) #np.percentile(data.flux[mask2], 50)*np.ones(data.time[mask2].shape[0])
    m2 = bruce.ambiguous_period.mono_event(duos['t_zero_2'][i], duos['width_2'][i], duos['depth_2'][i], data_event_2, name='TIC-{:}'.format(duos['tic_id'][i]), median_bin_size = None,convolve_bin_size = 3)

    # Fit the event and report plots
    fig_initial, ax_initial, fig_final, ax_final, return_data_2 = m2.fit_event_with_fixed_period(fit_period=30., plot=True, )
    fig_initial.tight_layout()
    fig_final.tight_layout()
    fig_initial.savefig(output_dir + '/' + 'TIC-{:}_EVENT_2_INITIAL_INITIAL.png'.format(duos['tic_id'][i]))
    fig_final.savefig(output_dir + '/' + 'TIC-{:}_EVENT_2_INITIAL_FINAL.png'.format(duos['tic_id'][i]))
    plt.close(fig_initial); plt.close(fig_final)

    # We are going to make a nice plot of the two events with their models
    fig, ax = plt.subplots(1,2, gridspec_kw={'hspace' : 0, 'wspace' : 0}, figsize = (6.4, 3.8))
    ax[0].errorbar(return_data_1[0], return_data_1[1], yerr=return_data_1[2], fmt='k.', alpha = 0.1)
    ax[0].plot(return_data_1[3], return_data_1[4], c='orange')
    ax[1].errorbar(return_data_2[0], return_data_2[1], yerr=return_data_2[2], fmt='k.', alpha = 0.1)
    ax[1].plot(return_data_2[3], return_data_2[4], c='orange')
    ax[1].set(yticks=[])
    ylim1 = ax[0].get_ylim()
    ylim2 = ax[1].get_ylim()
    ylim = [min(ylim1[0],ylim2[0]), max(ylim1[1], ylim2[1])]
    ax[0].set_ylim(ylim)
    ax[1].set_ylim(ylim)
    fig.supxlabel('Time from Transit [d]', fontsize=18, x=0.55, y = -0.005)
    fig.supylabel('Flux', fontsize=18)
    fig.suptitle(m2.name, y=0.95, x=0.55, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3', alpha=1.0), ha='center', fontsize=18)
    plt.subplots_adjust(right=0.99, top=0.99, bottom=0.13)
    fig.savefig(output_dir + '/' + 'TIC-{:}_BOTH_EVENTS.png'.format(duos['tic_id'][i]))
    plt.close(fig)

    ########################################################
    # CREATE THE AMBIGUOUS PERIOD OBJECT
    ########################################################
    p = bruce.ambiguous_period.ambiguous_period(data, events=[m1,m2], name='TIC-{:}'.format(duos['tic_id'][i]),
                        median_bin_size = 2,convolve_bin_size = 2)

    # Now mask and filter 
    p.mask_and_filter_events()

    # Calculate aliases
    # Do not use nsolutions_events here (that is superceeded later)
    nsolutions_events = p.calcualte_aliases(dx_lim=0.03, min_period=15)

    # Now calcualte whether we saw a transit by comparing the model to a flat line
    delta_L_data = p.calcualte_data_delta_L(data)
    
    
    ########################################################
    # CHECK NGTS PHOTOMETRY
    ########################################################
    ngts_data = []
    ngts_data_labels = []
    if os.path.isfile('ngts_data/TIC-{:}.fits'.format(duos['tic_id'][i])):
        ngts = Table.read('ngts_data/TIC-{:}.fits'.format(duos['tic_id'][i]), hdu=4)
        ngts.sort('BJD')
        mask = (ngts['SIGMA_XS']<0.02) & ~np.isnan(np.array(ngts['FLUX_SYSREM_ALIGNED'], dtype=np.float64)) &\
                     ~np.isnan(np.array(ngts['FLUX_ERR'], dtype=np.float64)) &\
                     ~np.isinf(np.array(ngts['FLUX_SYSREM_ALIGNED'], dtype=np.float64)) &\
                     ~np.isinf(np.array(ngts['FLUX_ERR'], dtype=np.float64))
        ngts = ngts[mask]
        
        # Now lets bin
        t_bin, f_bin, fe_bin, c = bruce.data.bin_data(np.array(ngts['BJD'], dtype=np.float64),
                                                  np.array(ngts['FLUX_SYSREM_ALIGNED'], dtype=np.float64),
                                                  0.5/24/3)

        t_bin, f_bin, fe_bin = t_bin[c>10], f_bin[c>10], fe_bin[c>10]
        if len(t_bin)>3 : 
            
            #######################################################################################
            #    OK, there are two ways of doing this based on data volume
            # We could 
            # 1. Split each night into its own LC so we can see how ruling out aliases
            #    progresses. This can be cumbersome and make large plots.
            # 2. Treat the NGTS LC as one, but if wee see a transit we will have 
            #    to do a retrospective calculation to see what aliases is was compatible with.
            #    It also might bork the delta_L calculation if a transit is seen but later excluded.
            #    Furthermore, if we only have a short amount of in-transit data, we have to be
            #    careful about how we detrend.
            #######################################################################################

            
#             #######################################################################################
#             # Aproach 1
#             #######################################################################################

#             # Now create the data products 
#             for seg in bruce.data.find_nights_from_data(t_bin, dx_lim=0.2): 
#                 # Now check the phase is consistent with one of the aliases

#                 # Now get the phases of all aliases
#                 phases = np.zeros((p.aliases.shape[0], len(np.array(t_bin, dtype=np.float64))))
#                 for j in range(len(phases)):
#                     phases[j] = bruce.data.phase_times(t_bin, p.events[0].de_get_epoch(), p.max_period/p.aliases[j], phase_offset=0.2)
#                 phase_widths = p.events[0].de_transit_width() / (p.max_period/p.aliases)

#                 # Now mask the data which doesent fall within 1 width of any alias
#                 useful_data = np.abs(phases) < (phase_widths[:,np.newaxis]/2) # Make for a nice plot

#                 # If so, lets make a data object for the night 
#                 if useful_data.any():
#                     # Now create the time series object
#                     d = bruce.ambiguous_period.photometry_time_series(t_bin[seg], f_bin[seg], fe_bin[seg])
#                     d.w = np.ones(len(d.time))

#                     # Now normalise to the median of the top 20%
#                     try:
#                         quarter_edges = np.linspace(np.min(d.time)-1e-3,np.max(d.time)+1e-3,5)
#                         dig = np.digitize(d.time, quarter_edges, right=True)
#                         medians = np.array([np.nanmedian(d.flux[dig==j]) for j in range(1,len(quarter_edges))])
#                         constant = np.nanmax(medians)
#                     except :  constant = np.nanmedian(d.flux)


#                     d.flux = d.flux / constant
#                     d.flux_err = d.flux_err / constant
#                     ngts_data.append(d)

#                     ngts_data_labels.append(Time(d.time[0], format='jd').datetime.strftime('%Y-%m-%d'))            
#                     print(Time(d.time[0], format='jd').datetime.strftime('%Y-%m-%d'))


            #######################################################################################
            # Aproach 2
            #######################################################################################
            # Now create the time series object
            d = bruce.ambiguous_period.photometry_time_series(t_bin, f_bin, fe_bin)
            d.w = np.ones(len(d.time))
            
            # We will do a crude variability removal
            # BE CAREFUL, only a small amount of in-transit data will 
            for seg in bruce.data.find_nights_from_data(d.time, dx_lim=0.2): 
                d.w[seg] = np.nanmedian(d.flux[seg])*np.ones(seg.shape[0])
            ngts_data.append(d)
            ngts_data_labels.append('NGTS data')
            
        # Now we need to adjust the time for TTV cases
        if duos['tic_id'][i]==118339710:
            for d in ngts_data:
                #print(len(d.time), d.time)
                #print(np.polyval(p, d.time))
                #d.time = d.time - np.polyval(p_ttv, d.time)
                d.time = d.time + p_ttv(d.time)
    
    ########################################################
    # CALCULATE DELTA L
    ########################################################
    #delta_L_data = delta_L_data + 100 # THIS FUDGE IS OFTEN NEEDED
#     for j in range(delta_L_data.shape[0]):
#         print(j, delta_L_data[j])
    delta_L_data_from_other_sectors_or_others = [p.calcualte_data_delta_L(j) for j in data_other_sectors]
    delta_L_data_from_ngts = [p.calcualte_data_delta_L(j) for j in ngts_data]
    p.delta_L = np.array([delta_L_data, *delta_L_data_from_other_sectors_or_others, *delta_L_data_from_ngts])

    ########################################################
    # Plot the aliases
    ########################################################
    fig, ax  = p.plot_aliases(phot_data=[*data_other_sectors, *ngts_data], 
                              phot_data_labels=[*data_labels, *ngts_data_labels])
    fig.savefig(output_dir + '/' + 'TIC-{:}_ALIASES.png'.format(duos['tic_id'][i]), dpi=300)
    plt.close(fig)
    
    ########################################################
    # Now report the aliases
    ########################################################
    aliases = (p.aliases)[p.alias_mask[:,-1]==p.alias_mask.max()]
    periods = (p.max_period/p.aliases)[p.alias_mask[:,-1]==p.alias_mask.max()]
    alaises = Table()
    alaises.add_column(Column(duos['tic_id'][i]*np.ones(len(aliases), dtype=int), name='tic_id'))
    alaises.add_column(Column(m1.de_get_epoch()*np.ones(len(aliases)), name='t_zero_1'))
    alaises.add_column(Column(m1.de_get_radius_1()*np.ones(len(aliases)), name='radius_1_1'))
    alaises.add_column(Column(m1.de_get_k()*np.ones(len(aliases)), name='k_1'))
    alaises.add_column(Column(m1.de_get_b()*np.ones(len(aliases)), name='b_1'))
    alaises.add_column(Column(m1.de_transit_width()*np.ones(len(aliases)), name='width_1'))
    alaises.add_column(Column(m2.de_get_epoch()*np.ones(len(aliases)), name='t_zero_2'))
    alaises.add_column(Column(m2.de_get_radius_1()*np.ones(len(aliases)), name='radius_1_2'))
    alaises.add_column(Column(m2.de_get_k()*np.ones(len(aliases)), name='k_2'))
    alaises.add_column(Column(m2.de_get_b()*np.ones(len(aliases)), name='b_2'))
    alaises.add_column(Column(m2.de_transit_width()*np.ones(len(aliases)), name='width_2'))
    alaises.add_column(Column(aliases, name='alias'))
    alaises.add_column(Column(periods, name='period'))
    alaises.write(output_dir + '/' + 'TIC-{:}_ALIASES.fits'.format(duos['tic_id'][i]), overwrite=True)

    
    ########################################################
    # Now plan the aliases
    ########################################################
    transit_events = p.transit_plan(start=Time.now(), end = Time.now()+TimeDelta(30, format='jd'), resolution = 1*u.minute,
                    tic_id=duos['tic_id'][i], observatory='Paranal',
                    sun_max_alt=-15, target_min_alt=30, moon_min_seperation=20,
                    min_time_in_transit=None, min_frac_in_transit=None)
    transit_events.add_column(Column(np.ones(len(transit_events))*m1.depth, name='transit_depth_1'))
    transit_events.add_column(Column(np.ones(len(transit_events))*m2.depth, name='transit_depth_2'))
    transit_events.add_column(Column(np.ones(len(transit_events))*m1.width, name='transit_width_1'))
    transit_events.add_column(Column(np.ones(len(transit_events))*m2.width, name='transit_width_2'))
    transit_events.write(output_dir + '/' + 'TIC-{:}_ALIASES_WINDOWS_PARANAL.fits'.format(duos['tic_id'][i]), overwrite=True)
    p.plot_all_events(transit_events, output_dir=output_dir)

    
    plt.close()
    gc.collect()


import multiprocess
with multiprocess.Pool(4) as p : p.map(func, range(len(duos)))
    


            
            
# f = open('failed.txt', 'w+')
# f.close()
# for i in range(len(duos))[5:]: 
#     try : func(i)
#     except: 
#         with open('failed.txt', 'a+') as f:
#             f.write('{:}\n'.format(duos['tic_id'][i]))

            
# print(np.argwhere(duos['tic_id']==126565211)[0][0])   
#func( np.argwhere(duos['tic_id']==140215502)[0][0] )




📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpzub4bif0


📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpzyvv70lp


📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmppg6u_lt7


📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpdhwr5hmc

Querying TIC 20904104 from MAST...
Querying TIC 140215502 from MAST...
Querying TIC 159490807 from MAST...Querying TIC 71028120 from MAST...

 [Done]


 [Done]


 [Done]



✅ Download summary:
Sector ...
------ ...
     5 ...
    32 ...

Temporary data stored in: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpzub4bif0
This directory will be deleted when the program exits.Downloading URL https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:TESS/product/tess2021118034608-s0038-0000000159490807-0209-s_lc.fits to ./mastDownload/TESS/tess2021118034608-s0038-0000000159490807-0209-s/tess2021118034608-s0038-0000000159490807-0209-s_lc.fits ...
W :  143
W :  144



✅ Download summary:
Sector ...
------ ...
     3 ...
    29 ...

Temporary data stored in: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpdhwr5hmc
This directory will be deleted when the program exits.
W :  W : 144 
144
W : W :   144144

 [Done]


...

No transists visible

📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpxvsm6k4t

Querying TIC 25608554 from MAST...


/Users/sam/anaconda3/lib/python3.9/site-packages/bruce-1.0.0-py3.9-macosx-10.9-x86_64.egg/bruce/ambiguous_period/data_retrieval.py:292: RuntimeWarning: All-NaN slice encountered
  if np.nanmedian(corrected_flux)>100 :
/Users/sam/anaconda3/lib/python3.9/site-packages/bruce-1.0.0-py3.9-macosx-10.9-x86_64.egg/bruce/ambiguous_period/data_retrieval.py:319: RuntimeWarning: All-NaN slice encountered
  print('MEDIAN CHECK not met : ' + str(np.nanmedian(corrected_flux)))


MEDIAN CHECK not met : nan



✅ Download summary:
Sector ...
------ ...
     8 ...
     9 ...
    35 ...
    62 ...
    89 ...

Temporary data stored in: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpxvsm6k4t
This directory will be deleted when the program exits.
W :  144
W :  144
W :  10
W :  144
W :  91
W :  25
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
W :  144
Initial Chi-Sqaured : 1302.65 [red 29.61]
Fitted parameters for TIC-25608554:
t_zero : 2459994.257159894
radius_1 : 0.030952571646908317
k : 0.12987121045192032
b : 0.9772946861904701
Final Chi-Sqaured : 57.32 [red 1.30]
Initial Chi-Sqaured : 913.37 [red 20.76]
Fitted parameters for TIC-25608554:
t_zero : 2460737.375278241
radius_1 : 0.0279775485963617
k : 0.09777942355401187
b : 0.9268145078688145
Final Chi-Sqaured : 37.90 [red 0.86]
1 50 1 49.541207889808966 743.1181183471344 15 15.165675884635396


100%|████████████████████████████████████████████████████████████████████████| 49/49 [00:05<00:00,  8.26it/s]
/Users/sam/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/sam/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/sam/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/sam/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  war

In [5]:
data

NameError: name 'data' is not defined

# Now look for solved systems

In [6]:
import glob
files = glob.glob('*/TIC-*_ALIASES.fits')

os.system('mkdir -p solved_systems; rm solved_systems/*')

for i in range(len(files)):
    tic_id  = int(files[i].split('/')[0])
    t = Table.read(files[i])
    if len(t)==1:
        os.system('cp {:}/TIC-{:}_ALIASES.png solved_systems'.format(tic_id,tic_id))
        
        print('{:}, {:}, {:}'.format(tic_id,  t['t_zero_1'][0], t['period'][0]))
        

# Now look for ones with no aliases left (to debug)

In [ ]:
import glob
files = glob.glob('*/TIC-*_ALIASES.fits')

os.system('mkdir -p no_aliases; rm no_aliases/*')

for i in range(len(files)):
    tic_id  = int(files[i].split('/')[0])
    t = Table.read(files[i])
    if len(t)==0:
        os.system('cp {:}/TIC-{:}_ALIASES.png no_aliases'.format(tic_id,tic_id))
        print(tic_id)

# Now look for ones that did not predict

In [10]:
for i in range(len(duos)):
    if not os.path.isfile('{:}/TIC-{:}_ALIASES_WINDOWS_PARANAL.fits'.format(duos['tic_id'][i], duos['tic_id'][i])):
        print(duos['tic_id'][i])

# Now make predictions about when they will transit

In [11]:
from astropy.table import Table, vstack
import glob

# Read all files and combine them
files = glob.glob('*/TIC-*_ALIASES_WINDOWS_PARANAL.fits')
events = vstack([Table.read(f) for f in files], join_type='outer')

# Group by tic_id and night
grouped = events.group_by(['tic_id', 'night'])

rows = []
for g in grouped.groups:
    # Start with the first row's values for all columns
    row = {col: g[col][0] for col in events.colnames}

    # Combine aliasP and aliasPer values as comma-separated strings
    row['aliasP'] = ','.join(str(v) for v in g['aliasP'])
    row['aliasPer'] = ','.join('{:.5f}'.format(v) for v in g['aliasPer'])
    
    rows.append(row)

# Create summary table
summary = Table(rows)
summary.sort('night')
print(summary['night', 'tic_id', 'aliasP', 'aliasPer'])

  night      tic_id  aliasP      aliasPer    
---------- --------- ------ -----------------
2025-11-04 332697924  14,29 48.83133,24.41566
2025-11-04 140750416     42          17.22305
2025-11-04 176518126  11,23 60.87333,30.43667
       ...       ...    ...               ...
2025-12-04 140750416  17,35 41.14396,20.57198
2025-12-04  95768667     18          37.43847
2025-12-04  42428568     22          32.54938
Length = 140 rows


# Now plot for a night

In [15]:
night = '2025-11-05'

t = summary[summary['night']==night]

p = bruce.ambiguous_period.ambiguous_period('a', events=['a','a'], name='dummy',median_bin_size = 2,convolve_bin_size = 2)
p.plot_all_events(t, output_dir='transit_plan')

In [18]:
t['night', 'tic_id', 'aliasP', 'aliasPer', 'transit_depth_1', 'transit_width_1']

night,tic_id,aliasP,aliasPer,transit_depth_1,transit_width_1
str10,int64,str41,str136,float64,float64
2025-11-05,42428568,16,44.03739,0.0052972436,0.39174843
2025-11-05,332657786,39,36.55523,0.03758840488,0.1244763641
2025-11-05,304339227,16,42.04048,0.0008614063,0.24149261
2025-11-05,251057075,28,25.09358,0.0023933053,0.15875384
2025-11-05,140750416,"19,39","37.02957,18.51478",0.009284496,0.2340945


In [16]:
t

tic_id,observatory,sun_max_alt,target_min_alt,moon_min_seperation,night,aliasP,aliasPer,alias_epoch,time_in_transit,frac_in_transit,moon_mean_sep,cycle_number,transit_center,transit_width,transit_depth_1,transit_depth_2,transit_width_1,transit_width_2
int64,str7,float64,float64,float64,str10,str41,str136,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64
42428568,Paranal,-15.0,30.0,20.0,2025-11-05,16,44.03739,2458475.6898154435,0.18888888888888888,0.4770431824343615,52.94387546931445,57,2460985.821183731,0.39595763202187456,0.0052972436,0.0048009553,0.39174843,0.3060124
332657786,Paranal,-15.0,30.0,20.0,2025-11-05,39,36.55523,2458536.7687440887,0.011111111111111112,0.047529511541421814,104.61297620697434,67,2460985.9692344572,0.23377288658705897,0.03758840488,0.04099340389,0.1244763641,0.1315262658
304339227,Paranal,-15.0,30.0,20.0,2025-11-05,16,42.04048,2458673.336292156,0.029166666666666667,0.11337040781628449,125.70080648683543,55,2460985.56288353,0.25726878140838066,0.0008614063,0.00047564507,0.24149261,0.1928184
251057075,Paranal,-15.0,30.0,20.0,2025-11-05,28,25.09358,2458426.3182872627,0.07291666666666667,0.36094111111027394,32.98756381611994,102,2460985.8630433334,0.2020181808671535,0.0023933053,0.0024456978,0.15875384,0.15350468
140750416,Paranal,-15.0,30.0,20.0,2025-11-05,"19,39","37.02957,18.51478",2458504.804306882,0.15486111111111112,0.8598785965562106,79.80114275077958,67,2460985.785256924,0.18009648307484974,0.009284496,0.009329319,0.2340945,0.1492758
